In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)


In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.11.0
0.12.0


In [3]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels = 6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels = 12,kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4,out_features=12)
        self.fc2 = nn.Linear(in_features=12,out_features=60)
        self.out = nn.Linear(in_features=60,out_features=10)

    def forward(self,t):
        #t = t #第一层输入
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size =2,stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t,kernel_size = 2,stride =2)

        t = t.reshape(-1,12*4*4)
        t = F.relu(self.fc1(t))

        t = F.relu(self.fc2(t))

        t = self.out(t)

        return t


In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/'
    ,train=True
    ,download=True
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

## 训练单个batch

In [6]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100)
optimizer = optim.Adam(network.parameters(),lr = 0.01)

batch = next(iter(train_loader))
images,labels = batch

preds = network(images)
loss = F.cross_entropy(preds,labels)

loss.backward()#计算梯度
optimizer.step()#更新权重

#-----------------------
print('loss1:',loss.item())
preds = network(images)
loss = F.cross_entropy(preds,labels)
print('loss2:',loss.item())


loss1: 2.32255220413208
loss2: 2.304868698120117


## 训练一个epoch

In [7]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer = optim.Adam(network.parameters(),lr = 0.01)

total_loss = 0
total_correct = 0

for batch in train_loader:
    images,labels = batch
    
    preds = network(images)
    loss = F.cross_entropy(preds,labels)
    #------------要将梯度归零 因为pytorch会累加梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds,labels)
print("epoch:",0,"total_correct",total_correct," loss:",total_loss)

epoch: 0 total_correct 46347  loss: 361.02063778042793


In [8]:
total_correct / len(train_set)

0.77245

## 训练多个epochs

In [10]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer = optim.Adam(network.parameters(),lr = 0.01)

total_loss = 0
total_correct = 0

for epoch in range(5):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        images,labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        #------------要将梯度归零 因为pytorch会累加梯度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
    print("epoch:",epoch,"total_correct",total_correct," loss:",total_loss)

epoch: 0 total_correct 46555  loss: 353.2511367201805
epoch: 1 total_correct 51081  loss: 244.41576735675335
epoch: 2 total_correct 51702  loss: 227.09699375927448
epoch: 3 total_correct 52065  loss: 216.49711048603058
epoch: 4 total_correct 52252  loss: 213.4748505204916


In [12]:
total_correct/len(train_set)

0.8708666666666667